In [5]:
from pathlib import Path
import numpy as np
import os

from FKMC.montecarlo import *

slurm_job_id = int(os.getenv('SLURM_ARRAY_JOB_ID', -1))
slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', 22))
max_slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_MAX', 24))
working_dir = Path(f".")

from time import time, sleep
sleeptime = np.random.random() * (5*60)
if slurm_job_id != -1: sleep(sleeptime)

Js = np.linspace(0.001, 10.0, 5)
Ts = np.linspace(0.001, 10.0, 5)
Ns = [4,8,16,32,64,128]

print(f'''len(Ts) = {len(Ts)},
      len(Js) = {len(Js)},
      prod = {len(Ts)*len(Js)},
      max_slurm_task_id = {max_slurm_task_id}''')

assert(max_slurm_task_id == len(Ts) * len(Js) - 1)
J = Js[slurm_task_id // len(Ts)]
T = Ts[slurm_task_id % len(Ts)]

logs = np.empty(shape = len(Ns), dtype = object)

for i, N in enumerate(Ns):
    parameters = dict(t = 1, alpha = 1.5, mu = 0, beta = 1/T, J = J, U = 5, normalise = True)
    MCMC_params = dict(
            state = np.arange(N, dtype = np.float64) % 2, #starting from a CDW state
            N_steps = int(10),
            #N_steps = int(1000 * 1000),
            N_burn_in = int(0), 
            #N_burn_in = int(10 * 1000), 
            #thin = 1000,
            logger = Eigenspectrum_IPR_all(bins = 1000, limit = 10),
            proposal = p_multi_site_uniform_reflect,
            accept_function = perturbation_accept,
            warnings = True,
        )

    print(f'starting N = {N}')
    t0 = time()
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
    logs[i].time = time() - t0

    parameters['J_matrix'] = '...'
    MCMC_params['state'] = '...'
    print(f'''
    params = {parameters},
    mc_params = {MCMC_params}
    ''')

filename = f'{slurm_task_id}.npz'

print(f'Total time {time() - t0:.0f}s')

if slurm_job_id != -1: 
    print(f'saving in {filename}')
    np.savez_compressed(working_dir/filename, 
        Ns = Ns, Ts = Ts, parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True,
        desc = ''
        )
print('done')

len(Ts) = 5,
      len(Js) = 5,
      prod = 25,
      max_slurm_task_id = 24
starting N = 4

    params = {'t': 1, 'alpha': 1.5, 'mu': 0, 'beta': 0.19998000199979998, 'J': 10.0, 'U': 5, 'normalise': True, 'J_matrix': '...'},
    mc_params = {'state': '...', 'N_steps': 1000, 'N_burn_in': 0, 'logger': <FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x7fb9c554d400>, 'proposal': <function p_multi_site_uniform_reflect at 0x7fb9c5c968c8>, 'accept_function': <function perturbation_accept at 0x7fb9c5c96bf8>, 'warnings': True}
    
starting N = 8

    params = {'t': 1, 'alpha': 1.5, 'mu': 0, 'beta': 0.19998000199979998, 'J': 10.0, 'U': 5, 'normalise': True, 'J_matrix': '...'},
    mc_params = {'state': '...', 'N_steps': 1000, 'N_burn_in': 0, 'logger': <FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x7fb9c554d7f0>, 'proposal': <function p_multi_site_uniform_reflect at 0x7fb9c5c968c8>, 'accept_function': <function perturbation_accept at 0x7fb9c5c96bf8>, 'warnings': True}
    
starting N = 16


0.4444444444444444